In [9]:
import pandas as pd
import numpy as np
import torch
from config import basic_conf as conf
conf.DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
from libs import ModelManager as mm
from config.constants import HyperParamKey

import logging
import itertools

import matplotlib.pyplot as plt
from importlib import reload

%matplotlib inline

In [2]:
conf.init_logger(logging.INFO, logfile=None)
mgr = mm.ModelManager(mode='notebook')

hparams = {
    HyperParamKey.NUM_EPOCH: 1,
    HyperParamKey.BATCH_SIZE: 2048
}
mgr.hparams.update(hparams)

[2018-10-27 20:27:28] [INFO] Initializing Model Manager, version 0.4.0 ...
[2018-10-27 20:27:28] [INFO] 
=== Models Available ===
BagOfWords
NLIRNN
[2018-10-27 20:27:28] [INFO] 
=== Loaders Available ===
IMDB
SNLI
[2018-10-27 20:27:28] [INFO] 
*********** Model Manager Details ***********
-- self.hparams.num_epochs = 10
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.5
-- self.hparams.dropout_fc = 0.5
-- self.hparams.batch_size = 256
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 50
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 10
-- self.hparams.es_req_prog = 0.01
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <class 'torch.optim.adam.Adam'>
-- self.hparams.scheduler = <class 'torch.optim.lr_scheduler.ExponentialLR'>
-- self.hparams.scheduler_gamma = 0.95
-- self.hparams.crite

In [3]:
print(mgr.manager_details)


*********** Model Manager Details ***********
-- self.hparams.num_epochs = 1
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.5
-- self.hparams.dropout_fc = 0.5
-- self.hparams.batch_size = 2048
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 50
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 10
-- self.hparams.es_req_prog = 0.01
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <class 'torch.optim.adam.Adam'>
-- self.hparams.scheduler = <class 'torch.optim.lr_scheduler.ExponentialLR'>
-- self.hparams.scheduler_gamma = 0.95
-- self.hparams.criterion = <class 'torch.nn.modules.loss.CrossEntropyLoss'>
-- self.cparams.save_best_model = True
-- self.cparams.save_each_epoch = True
-- self.cparams.ignore_params = ['pre_trained_vecs']
-- self.cparams.nli_train_path = data/nli/snli_train.tsv

In [4]:
mgr.load_data(mm.loaderRegister.SNLI)

[2018-10-27 20:27:31] [INFO] Loading data using SNLI ...
[2018-10-27 20:27:31] [INFO] loading raw training data set ...
[2018-10-27 20:27:31] [INFO] loading raw training data set ...
[2018-10-27 20:27:31] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-27 20:27:44] [INFO] converting training set to index ...
[2018-10-27 20:27:45] [INFO] converting val set to index ...
[2018-10-27 20:27:45] [INFO] piping data into pytorch DataLoaders ...


In [12]:
lr_list = [0.01, 0.001]
decay_rates = [0.95, 0.8]
rnn_hidden_sizes = [50, 100, 200]
dropouts = [0.25, 0.5, 0.75]

def hparam_to_label(tp):
    label = 'mod'
    label += ' lrinv' + str(int(1 / tp[0]))
    label += ' decay' + str(int(tp[1] * 100))
    label += ' rhs' + str(int(tp[2]))
    label += ' drop' + str(int(tp[3] * 100))
    return label

param_count_dict = {}
for hp_tuple in list(itertools.product(lr_list, decay_rates, rnn_hidden_sizes, dropouts)):
    lr, decay, rnn_hidden, dropout = tuple(hp_tuple)
    hparam_overrides = {HyperParamKey.LR: lr,
                        HyperParamKey.SCHEDULER_GAMMA: decay,
                        HyperParamKey.RNN_HIDDEN_SIZE: rnn_hidden,
                        HyperParamKey.DROPOUT_FC: dropout,
                        HyperParamKey.DROPOUT_RNN: dropout}

    mgr.hparams.update(hparam_overrides)
    mgr.new_model(mm.modelRegister.NLIRNN, label=hparam_to_label(hp_tuple), nolog=True)
    param_count_dict[hparam_to_label(hp_tuple)] = mgr.model.output_dict['total_param_count']
    mgr.dump_model()


[2018-10-27 20:30:06] [INFO] New Model initialized: /mod lrinv100 decay95 rhs50 drop25, all model output files will be saved here: model_saves/modlrinv100decay95rhs50drop25/
[2018-10-27 20:30:06] [INFO] New Model initialized: /mod lrinv100 decay95 rhs50 drop50, all model output files will be saved here: model_saves/modlrinv100decay95rhs50drop50/
[2018-10-27 20:30:07] [INFO] New Model initialized: /mod lrinv100 decay95 rhs50 drop75, all model output files will be saved here: model_saves/modlrinv100decay95rhs50drop75/
[2018-10-27 20:30:07] [INFO] New Model initialized: /mod lrinv100 decay95 rhs100 drop25, all model output files will be saved here: model_saves/modlrinv100decay95rhs100drop25/
[2018-10-27 20:30:07] [INFO] New Model initialized: /mod lrinv100 decay95 rhs100 drop50, all model output files will be saved here: model_saves/modlrinv100decay95rhs100drop50/
[2018-10-27 20:30:08] [INFO] New Model initialized: /mod lrinv100 decay95 rhs100 drop75, all model output files will be saved 

In [13]:
param_count_dict

{'mod lrinv100 decay95 rhs50 drop25': 126003,
 'mod lrinv100 decay95 rhs50 drop50': 126003,
 'mod lrinv100 decay95 rhs50 drop75': 126003,
 'mod lrinv100 decay95 rhs100 drop25': 281603,
 'mod lrinv100 decay95 rhs100 drop50': 281603,
 'mod lrinv100 decay95 rhs100 drop75': 281603,
 'mod lrinv100 decay95 rhs200 drop25': 682803,
 'mod lrinv100 decay95 rhs200 drop50': 682803,
 'mod lrinv100 decay95 rhs200 drop75': 682803,
 'mod lrinv100 decay80 rhs50 drop25': 126003,
 'mod lrinv100 decay80 rhs50 drop50': 126003,
 'mod lrinv100 decay80 rhs50 drop75': 126003,
 'mod lrinv100 decay80 rhs100 drop25': 281603,
 'mod lrinv100 decay80 rhs100 drop50': 281603,
 'mod lrinv100 decay80 rhs100 drop75': 281603,
 'mod lrinv100 decay80 rhs200 drop25': 682803,
 'mod lrinv100 decay80 rhs200 drop50': 682803,
 'mod lrinv100 decay80 rhs200 drop75': 682803,
 'mod lrinv1000 decay95 rhs50 drop25': 126003,
 'mod lrinv1000 decay95 rhs50 drop50': 126003,
 'mod lrinv1000 decay95 rhs50 drop75': 126003,
 'mod lrinv1000 dec

In [ ]:
mgr.train()

In [ ]:
mgr.get_results().T

In [ ]:
mgr.graph_training_curves()